<a href="https://colab.research.google.com/github/yohangumiel/Aulas-PUC-PR/blob/main/Agentes%20Conversacionais/Aula_1_Automatiza%C3%A7%C3%A3o_de_pedidos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Exemplo simples de automatização de pedidos**.

## Adaptação voltada a parte de interpretação do texto para um pedido. O código original (feito pelo Prof. Lucas Oliveira) que inclusive apresenta elementos de transcrição de aúdio para texto, está disponível em https://colab.research.google.com/drive/192ttdc_MvwhGtP-_hkTVnRam8eAxBnJP?usp=sharing

## Código desenvolvido por Prof. [Lucas Oliveira](https://www.linkedin.com/in/lucassilvaoliveira/) (PUC-PR) e Prof. [Yohan Gumiel](https://www.linkedin.com/in/yohan-gumiel-93b33b95/) (PUC-PR/UFMG) 



## Contexto geral

Uma grande rede de fastfood deseja automatizar completamente seus processos, desde atendimento ao cliente até a produção de seus lanches. O cliente fará seu pedido em painéis de auto-atendimento utilizando sua voz, e o pedido deve ser passado às máquinas que produzem e selecionam os produtos para entrega.

A interface das máquinas prevê a leitura de um arquivo texto no seguinte formato:
QUANTIDADE;PRODUTO;TAMANHO

Defina um algoritmo que obtenha o pedido do usuário, e escreva em um arquivo-texto os produtos que devem ser preparados.

In [ ]:
import nltk
from nltk.corpus import mac_morpho
from nltk.tag import tnt
from nltk import tokenize

nltk.download('mac_morpho')

[nltk_data] Downloading package mac_morpho to /root/nltk_data...
[nltk_data]   Package mac_morpho is already up-to-date!


True

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tagged_sents = mac_morpho.tagged_sents()

In [ ]:
tnt_pos = tnt.TnT()
tnt_pos.train(tagged_sents)

In [ ]:
pedido = 'Olá eu gostaria de duas batatas médias um refrigerante grande e um sanduíche pequeno'


def get_order(pedido):

    tagged_text = tnt_pos.tag(tokenize.word_tokenize(pedido, language='portuguese'))
    print(tagged_text)

    c = 0
    for token in tagged_text:
        print(token)
        produto = ''
        quantidade = '1'
        tamanho = 'médio'
        
        # Se encontrou substantivo (Produto)
        if token[1] == 'N':
            
            produto = token[0]
        
            prevTok = tagged_text[c - 1]
            nextTok = tagged_text[c + 1]
            
            if prevTok[1] == 'NUM':
                quantidade = prevTok[0]
                
            if nextTok[1] == 'ADJ':
                tamanho = nextTok[0]
                
            pedido_interface = quantidade + " ; " + produto + " ; " + tamanho
                       
            print('\nPEDIDO:', pedido_interface, '\n')
            
        c += 1

get_order(pedido)

[('Olá', 'IN'), ('eu', 'PROPESS'), ('gostaria', 'V'), ('de', 'PREP'), ('duas', 'NUM'), ('batatas', 'N'), ('médias', 'ADJ'), ('um', 'ART'), ('refrigerante', 'N'), ('grande', 'ADJ'), ('e', 'KC'), ('um', 'ART'), ('sanduíche', 'N'), ('pequeno', 'ADJ')]
('Olá', 'IN')
('eu', 'PROPESS')
('gostaria', 'V')
('de', 'PREP')
('duas', 'NUM')
('batatas', 'N')

PEDIDO: duas ; batatas ; médias 

('médias', 'ADJ')
('um', 'ART')
('refrigerante', 'N')

PEDIDO: 1 ; refrigerante ; grande 

('grande', 'ADJ')
('e', 'KC')
('um', 'ART')
('sanduíche', 'N')

PEDIDO: 1 ; sanduíche ; pequeno 

('pequeno', 'ADJ')


## **Primeiro exercício**: analisar resultado do POS-tag (classificação gramatical), o que não reconhece e o que poderia ter problemas reconhecendo?

#### **Resposta**: elementos como "coca" e sanduíches como "big mac" são tokens UNK (fora do vocabulário), ou seja, o pos-tagger não os entende como substantivos devido ao seu contexto limitado. Adicionalmente, "uma" é considerado um artigo e não um número.



In [ ]:
pedido = 'Olá eu gostaria de um guaraná uma coca duas batatas fritas médias e quatro big macs'

get_order(pedido)


[('Olá', 'IN'), ('eu', 'PROPESS'), ('gostaria', 'V'), ('de', 'PREP|+'), ('um', 'ART'), ('guaraná', 'N'), ('uma', 'ART'), ('coca', 'Unk'), ('duas', 'NUM'), ('batatas', 'N'), ('fritas', 'PCP'), ('médias', 'ADJ'), ('e', 'KC'), ('quatro', 'NUM'), ('big', 'N|EST'), ('macs', 'Unk')]
('Olá', 'IN')
('eu', 'PROPESS')
('gostaria', 'V')
('de', 'PREP|+')
('um', 'ART')
('guaraná', 'N')

PEDIDO: 1 ; guaraná ; médio 

('uma', 'ART')
('coca', 'Unk')
('duas', 'NUM')
('batatas', 'N')

PEDIDO: duas ; batatas ; médio 

('fritas', 'PCP')
('médias', 'ADJ')
('e', 'KC')
('quatro', 'NUM')
('big', 'N|EST')
('macs', 'Unk')


## **Segunda questão**: se fosse um sistema livre (sem reconhecimento de voz) e o texto fosse inteiro em maísculo ou os itens fossem escritos em maísculo, o que aconteceria?

#### **Resposta**: elementos como "Guaraná", "Coca" e "Big Mac" seriam considerados como substantivos próprio devido as letras maiúsculas.

#### **Resposta**: se o texto inteiro fosse em maiúsculo teriam vários casos de UNK (fora do vocabulário)


In [ ]:
pedido = 'Olá eu gostaria de um Guaraná uma Coca duas batatas fritas médias e quatro Big Macs'

get_order(pedido)

[('Olá', 'IN'), ('eu', 'PROPESS'), ('gostaria', 'V'), ('de', 'PREP|+'), ('um', 'ART'), ('Guaraná', 'NPROP'), ('uma', 'ART'), ('Coca', 'NPROP'), ('duas', 'NUM'), ('batatas', 'N'), ('fritas', 'PCP'), ('médias', 'ADJ'), ('e', 'KC'), ('quatro', 'NUM'), ('Big', 'NPROP'), ('Macs', 'NPROP')]
('Olá', 'IN')
('eu', 'PROPESS')
('gostaria', 'V')
('de', 'PREP|+')
('um', 'ART')
('Guaraná', 'NPROP')
('uma', 'ART')
('Coca', 'NPROP')
('duas', 'NUM')
('batatas', 'N')

PEDIDO: duas ; batatas ; médio 

('fritas', 'PCP')
('médias', 'ADJ')
('e', 'KC')
('quatro', 'NUM')
('Big', 'NPROP')
('Macs', 'NPROP')


In [ ]:
pedido = 'Olá eu gostaria de um Guaraná uma Coca duas batatas fritas médias e quatro Big Macs'.upper()

get_order(pedido)

[('OLÁ', 'Unk'), ('EU', 'Unk'), ('GOSTARIA', 'Unk'), ('DE', 'NPROP'), ('UM', 'Unk'), ('GUARANÁ', 'Unk'), ('UMA', 'Unk'), ('COCA', 'Unk'), ('DUAS', 'Unk'), ('BATATAS', 'Unk'), ('FRITAS', 'Unk'), ('MÉDIAS', 'Unk'), ('E', 'KC'), ('QUATRO', 'Unk'), ('BIG', 'Unk'), ('MACS', 'Unk')]
('OLÁ', 'Unk')
('EU', 'Unk')
('GOSTARIA', 'Unk')
('DE', 'NPROP')
('UM', 'Unk')
('GUARANÁ', 'Unk')
('UMA', 'Unk')
('COCA', 'Unk')
('DUAS', 'Unk')
('BATATAS', 'Unk')
('FRITAS', 'Unk')
('MÉDIAS', 'Unk')
('E', 'KC')
('QUATRO', 'Unk')
('BIG', 'Unk')
('MACS', 'Unk')


## **Terceira questão**: Já vimos a abordagem acima não é robusta para sanduíches do McDonalds. Como poderia ser feita uma adaptação para achar determinados sanduíches? Ainda gostariamos de fazer uso do pos-tagger para achar determinados itens, como número de sanduíches e adjetivos (como especial).

#### **Resposta**: Poderia ser utilizado de expressões regulares, porém neste contexto seria interessante usar o pos-tagger para pegar números (quantidade) e adjetivos. Por este motivo foi utilizada uma estratégia de considerar sanduíches de múltiplas palavras como somente uma palavra/token e tentar achar estes tokens únicos durante a varredura. Não queremos depender do pos tagging  para marcar os sanduíches, porém queremos as unidades e adjetivos pelos pos tag.



#### **Para ajudar no código e dar um pouco de fome.**




![tabela de sanduiches](https://drive.google.com/uc?export=view&id=14ZKl4btWvwqM27mpcepVY00ETYOY6e78)



In [ ]:
pedido = 'Olá eu gostaria de um Guaraná uma Coca duas Batatas Médias e Quatro Big Macs e 2 big mac e quatro quarterão com queijo especial'


def get_order(pedido):

    pedido = pedido.lower()
    sanduiches = ['big mac','quarterão com queijo','quarteirão com queijo','cheddar mcmelt','duplo cheeseburger','triplo cheeseburger']

    list_sanduiches = ['big_mac','big_macs','quarterão_com_queijo','quarteirão_com_queijo','cheddar_mcmelt','duplo_cheeseburger','triplo_cheeseburger'] 
    # Não queremos depender do postagging para marcar os sanduíches, porém queremos as unidades e adjetivos pelo pos tag. Poderiamos refinar as regex (números), por exemplo: conversão de um para 1

    list_refrigerantes = ['coca','guaraná']

    for sanduiche in sanduiches:
        pedido = pedido.replace(sanduiche,'_'.join(sanduiche.split()))

    print(pedido)

    tagged_text = tnt_pos.tag(tokenize.word_tokenize(pedido, language='portuguese'))
    print(tagged_text)

    c = 0
    for token in tagged_text:
        print(token)
        produto = ''
        quantidade = ''
        tamanho = ''
        
        # Se encontrou substantivo (Produto)
        if token[1] == 'N' or  token[0] in list_sanduiches or token[0] in list_refrigerantes:
            
            produto = token[0]
        
            prevTok = tagged_text[c - 1]
            nextTok = tagged_text[c + 1]
            
            if prevTok[1] == 'NUM':
                quantidade = prevTok[0]
                
            if nextTok[1] == 'ADJ':
                tamanho = nextTok[0]
                
            pedido_interface = quantidade + " ; " + produto + " ; " + tamanho
                       
            print('\nPEDIDO:', pedido_interface, '\n')
            
        c += 1

get_order(pedido)



olá eu gostaria de um guaraná uma coca duas batatas médias e quatro big_macs e 2 big_mac e quatro quarterão_com_queijo especial
[('olá', 'Unk'), ('eu', 'PROPESS'), ('gostaria', 'V'), ('de', 'PREP|+'), ('um', 'ART'), ('guaraná', 'N'), ('uma', 'ART'), ('coca', 'Unk'), ('duas', 'NUM'), ('batatas', 'N'), ('médias', 'ADJ'), ('e', 'KC'), ('quatro', 'NUM'), ('big_macs', 'Unk'), ('e', 'KC'), ('2', 'NUM'), ('big_mac', 'Unk'), ('e', 'KC'), ('quatro', 'NUM'), ('quarterão_com_queijo', 'Unk'), ('especial', 'ADJ')]
('olá', 'Unk')
('eu', 'PROPESS')
('gostaria', 'V')
('de', 'PREP|+')
('um', 'ART')
('guaraná', 'N')

PEDIDO:  ; guaraná ;  

('uma', 'ART')
('coca', 'Unk')

PEDIDO:  ; coca ;  

('duas', 'NUM')
('batatas', 'N')

PEDIDO: duas ; batatas ; médias 

('médias', 'ADJ')
('e', 'KC')
('quatro', 'NUM')
('big_macs', 'Unk')

PEDIDO: quatro ; big_macs ;  

('e', 'KC')
('2', 'NUM')
('big_mac', 'Unk')

PEDIDO: 2 ; big_mac ;  

('e', 'KC')
('quatro', 'NUM')
('quarterão_com_queijo', 'Unk')

PEDIDO: quatro 

#### No trecho de código abaixo foi adicionada uma regra para considerar "uma" e "um" como números e não artigos

In [ ]:
pedido = 'Olá eu gostaria de um Guaraná uma Coca duas Batatas Médias e Quatro Big Macs e 2 big mac e quatro quarterão com queijo especial'


def get_order(pedido):

    pedido = pedido.lower()
    sanduiches = ['big mac','quarterão com queijo','quarteirão com queijo','cheddar mcmelt','duplo cheeseburger','triplo cheeseburger']

    list_sanduiches = ['big_mac','big_macs','quarterão_com_queijo','quarteirão_com_queijo','cheddar_mcmelt','duplo_cheeseburger','triplo_cheeseburger'] # não quero depender do postagging para marcar os sanduíches, porém quero as unidades pelo Postag, poderia refinar as regex (números), por exemplo: conversão de um para 1

    list_refrigerantes = ['coca','guaraná']

    for sanduiche in sanduiches:
        pedido = pedido.replace(sanduiche,'_'.join(sanduiche.split()))

    print(pedido)

    tagged_text = tnt_pos.tag(tokenize.word_tokenize(pedido, language='portuguese'))


    tagged_text = [list(elem) for elem in tagged_text]

    print(tagged_text)

    for token in list(tagged_text):
        if token[0] == 'um' or token[0] == 'uma':
            token[1] = 'NUM'

    print(tagged_text)

    c = 0
    for token in tagged_text:
        print(token)
        produto = ''
        quantidade = ''
        tamanho = ''
        
        # Se encontrou substantivo (Produto)
        if token[1] == 'N' or  token[0] in list_sanduiches or token[0] in list_refrigerantes:
            
            produto = token[0]
        
            prevTok = tagged_text[c - 1]
            nextTok = tagged_text[c + 1]
            
            if prevTok[1] == 'NUM':
                quantidade = prevTok[0]
                
            if nextTok[1] == 'ADJ':
                tamanho = nextTok[0]
                
            pedido_interface = quantidade + " ; " + produto + " ; " + tamanho
                       
            print('\nPEDIDO:', pedido_interface, '\n')

            
        c += 1

get_order(pedido)

olá eu gostaria de um guaraná uma coca duas batatas médias e quatro big_macs e 2 big_mac e quatro quarterão_com_queijo especial
[['olá', 'Unk'], ['eu', 'PROPESS'], ['gostaria', 'V'], ['de', 'PREP|+'], ['um', 'ART'], ['guaraná', 'N'], ['uma', 'ART'], ['coca', 'Unk'], ['duas', 'NUM'], ['batatas', 'N'], ['médias', 'ADJ'], ['e', 'KC'], ['quatro', 'NUM'], ['big_macs', 'Unk'], ['e', 'KC'], ['2', 'NUM'], ['big_mac', 'Unk'], ['e', 'KC'], ['quatro', 'NUM'], ['quarterão_com_queijo', 'Unk'], ['especial', 'ADJ']]
[['olá', 'Unk'], ['eu', 'PROPESS'], ['gostaria', 'V'], ['de', 'PREP|+'], ['um', 'NUM'], ['guaraná', 'N'], ['uma', 'NUM'], ['coca', 'Unk'], ['duas', 'NUM'], ['batatas', 'N'], ['médias', 'ADJ'], ['e', 'KC'], ['quatro', 'NUM'], ['big_macs', 'Unk'], ['e', 'KC'], ['2', 'NUM'], ['big_mac', 'Unk'], ['e', 'KC'], ['quatro', 'NUM'], ['quarterão_com_queijo', 'Unk'], ['especial', 'ADJ']]
['olá', 'Unk']
['eu', 'PROPESS']
['gostaria', 'V']
['de', 'PREP|+']
['um', 'NUM']
['guaraná', 'N']

PEDIDO: um ; g

## **Quarta questão**: Se usassemos outro pos tagger, como o Stanza, será que haveria alguma diferença?

#### **Resposta**: Neste contexto não haveria uma mudança muito significativa. Pelos testes feitos ambos tem dificuldades com o "um" e "uma" e com sanduíches específicos como "big mac".

In [ ]:
!pip install stanza

import stanza
stanza.download('pt')

     |████████████████████████████████| 342 kB 7.7 MB/s 


2021-08-17 19:34:16 INFO: Downloading default packages for language: pt (Portuguese)...


2021-08-17 19:34:54 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
stanza_pt = stanza.Pipeline('pt', tokenize_pretokenized=True, use_gpu= True)

2021-08-17 19:34:54 INFO: Loading these models for language: pt (Portuguese):
| Processor | Package |
-----------------------
| tokenize  | bosque  |
| mwt       | bosque  |
| pos       | bosque  |
| lemma     | bosque  |
| depparse  | bosque  |

2021-08-17 19:34:54 INFO: Use device: cpu
2021-08-17 19:34:54 INFO: Loading: tokenize
2021-08-17 19:34:54 INFO: Loading: mwt
2021-08-17 19:34:55 INFO: Loading: pos
2021-08-17 19:34:55 INFO: Loading: lemma
2021-08-17 19:34:55 INFO: Loading: depparse
2021-08-17 19:34:55 INFO: Done loading processors!


In [ ]:
def get_order(pedido):
    

    tagged_text = tnt_pos.tag(tokenize.word_tokenize(pedido, language='portuguese'))

    tagged_text_stanza = []

    doc = stanza_pt([tokenize.word_tokenize(pedido, language='portuguese')])              
    for sent in doc.sentences:
        for word in sent.words:
            tagged_text_stanza.append((word.text,word.pos))


    print('Mac-morpho:\n', tagged_text)
    print('Stanza:\n', tagged_text_stanza)


    c = 0
    for token in tagged_text_stanza:
        print(token)
        
        produto = ''
        quantidade = ''
        tamanho = ''
        
        # Se encontrou substantivo (Produto)
        if token[1] == 'NOUN':
            
            produto = token[0]
        
            prevTok = tagged_text_stanza[c - 1]
            nextTok = tagged_text_stanza[c + 1]
            
            if prevTok[1] == 'NUM':
                quantidade = prevTok[0]
                
            if nextTok[1] == 'ADJ':
                tamanho = nextTok[0]
                
            pedido_interface = quantidade + " ; " + produto + " ; " + tamanho 
            
            print('\nPEDIDO:', pedido_interface, '\n')
            
        c += 1

In [ ]:
pedido = 'Olá eu gostaria de um refrigerante pequeno duas batatas médias e quatro sanduíches pequenos'
get_order(pedido)

Mac-morpho:
 [('Olá', 'IN'), ('eu', 'PROPESS'), ('gostaria', 'V'), ('de', 'PREP|+'), ('um', 'ART'), ('refrigerante', 'N'), ('pequeno', 'ADJ'), ('duas', 'NUM'), ('batatas', 'N'), ('médias', 'ADJ'), ('e', 'KC'), ('quatro', 'NUM'), ('sanduíches', 'N'), ('pequenos', 'ADJ')]
Stanza:
 [('Olá', 'ADV'), ('eu', 'PRON'), ('gostaria', 'VERB'), ('de', 'ADP'), ('um', 'DET'), ('refrigerante', 'NOUN'), ('pequeno', 'ADJ'), ('duas', 'NUM'), ('batatas', 'NOUN'), ('médias', 'ADJ'), ('e', 'CCONJ'), ('quatro', 'NUM'), ('sanduíches', 'NOUN'), ('pequenos', 'ADJ')]
('Olá', 'ADV')
('eu', 'PRON')
('gostaria', 'VERB')
('de', 'ADP')
('um', 'DET')
('refrigerante', 'NOUN')

PEDIDO:  ; refrigerante ; pequeno 

('pequeno', 'ADJ')
('duas', 'NUM')
('batatas', 'NOUN')

PEDIDO: duas ; batatas ; médias 

('médias', 'ADJ')
('e', 'CCONJ')
('quatro', 'NUM')
('sanduíches', 'NOUN')

PEDIDO: quatro ; sanduíches ; pequenos 

('pequenos', 'ADJ')


In [ ]:
pedido = 'Olá eu gostaria de um guaraná uma coca duas batatas médias e quatro big macs e 2 big mac e quatro quarterão com queijo especial'

get_order(pedido)

Mac-morpho:
 [('Olá', 'IN'), ('eu', 'PROPESS'), ('gostaria', 'V'), ('de', 'PREP|+'), ('um', 'ART'), ('guaraná', 'N'), ('uma', 'ART'), ('coca', 'Unk'), ('duas', 'NUM'), ('batatas', 'N'), ('médias', 'ADJ'), ('e', 'KC'), ('quatro', 'NUM'), ('big', 'N|EST'), ('macs', 'Unk'), ('e', 'KC'), ('2', 'N'), ('big', 'ADJ|EST'), ('mac', 'Unk'), ('e', 'KC'), ('quatro', 'NUM'), ('quarterão', 'Unk'), ('com', 'PREP'), ('queijo', 'N'), ('especial', 'ADJ')]
Stanza:
 [('Olá', 'ADV'), ('eu', 'PRON'), ('gostaria', 'VERB'), ('de', 'ADP'), ('um', 'DET'), ('guaraná', 'NOUN'), ('uma', 'DET'), ('coca', 'NOUN'), ('duas', 'NUM'), ('batatas', 'NOUN'), ('médias', 'ADJ'), ('e', 'CCONJ'), ('quatro', 'NUM'), ('big', 'X'), ('macs', 'X'), ('e', 'CCONJ'), ('2', 'NUM'), ('big', 'X'), ('mac', 'X'), ('e', 'CCONJ'), ('quatro', 'NUM'), ('quarterão', 'VERB'), ('com', 'ADP'), ('queijo', 'NOUN'), ('especial', 'ADJ')]
('Olá', 'ADV')
('eu', 'PRON')
('gostaria', 'VERB')
('de', 'ADP')
('um', 'DET')
('guaraná', 'NOUN')
;guaraná;
('uma'

### Referências
* [Reconhecimento de Voz com Python: Faça seu primeiro Olá Mundo com Speech Recognition!](https://medium.com/brasil-ai/reconhecimento-voz-python-35a5023767ca)